In [1]:
%matplotlib ipympl
from QDP import qdp
import os.path
import numpy as np
from scipy.stats import poisson
import matplotlib.pyplot as plt
dp = '/home/ebert/jupyter-notebooks/FNode-data'

In [2]:
exp_name = '2018_05_08_15_04_02_rb-loading-cs'
if exp_name:
    exp_date = '_'.join(exp_name.split('_')[:3])
    experiment_file = os.path.join(exp_date, exp_name, 'results.hdf5')
else:
    experiment_file = ''
q = qdp.QDP(base_data_path=dp)
q.load_data_file(experiment_file)
print "ivars: {}".format(q.experiments[0]['variable_list'])
cs_cuts = [[141],[3]]
rb_cuts = [[86],[3]]
q.set_thresholds(cs_cuts, roi=0)
q.set_thresholds(rb_cuts, roi=1)
retention = q.apply_thresholds(exclude_rois=[2])
cuts = q.get_thresholds()
roi_map = ['cs', 'rb']
entries = np.product(retention['retention'].shape[:-1])
tbl_str = "ivar: {0:.4f}\tcloading: {1:.3f}\trloading: {2:.3f}\t\tcretention: {3:.3f} +- {4:.3f}\trretention: {5:.3f} +- {6:.3f}"
for i in range(entries):
    print(tbl_str.format(
        retention['ivar'].reshape((entries,-1))[i][0],
        retention['loading'].reshape((entries,-1))[i][0], retention['loading'].reshape((entries,-1))[i][1],
        retention['retention'].reshape((entries,-1))[i][0], retention['error'].reshape((entries,-1))[i][0],
        retention['retention'].reshape((entries,-1))[i][1], retention['error'].reshape((entries,-1))[i][1]
    ))
q.save_experiment_data()
xlab = q.experiments[0]['variable_desc'][q.experiments[0]['variable_list'][0]]['description']
fig, ax = plt.subplots()
for e in range(len(retention['ivar'])):
    ax.plot(retention['ivar'][e, :, 0], retention['loading'][e, :, 0], 'bo')
    ax.plot(retention['ivar'][e, :, 0], retention['loading'][e, :, 1], 'rx')
ax.set_ylim(0, 1.05*np.max(retention['loading']))
ax.set_xlabel(xlab)
ax.set_ylabel('loading')

fmts = ['pdf', 'png']
fn = os.path.join(dp, q.experiments[0]['source_path'], 'loading_scan.{}')
for fmt in fmts:
    fig.savefig(fn.format(fmt), dpi=200, format=fmt)

name 'fort_exp' is not defined
name 'rb_uwave_freq' is not defined
ivars: [u'rb_d2_mot_amp']
no loading observed
no loading observed
no loading observed
no loading observed
no loading observed
ivar: -8.6000	cloading: 0.210	rloading: 0.171		cretention: 0.738 +- 0.014	rretention: 0.947 +- 0.014
ivar: -9.0000	cloading: 0.226	rloading: 0.128		cretention: 0.748 +- 0.020	rretention: 0.922 +- 0.020
ivar: -9.4000	cloading: 0.277	rloading: 0.089		cretention: 0.874 +- 0.016	rretention: 0.966 +- 0.016
ivar: -9.8000	cloading: 0.179	rloading: 0.079		cretention: 0.872 +- 0.025	rretention: 0.924 +- 0.025
ivar: -10.2000	cloading: 0.192	rloading: 0.029		cretention: 0.943 +- 0.028	rretention: 0.966 +- 0.028
ivar: -10.6000	cloading: 0.203	rloading: 0.004		cretention: 0.842 +- 0.182	rretention: 0.750 +- 0.182


/home/ebert/jupyter-notebooks/FNode/analyses/qdp/QDP/qdp.py:102: RuntimeWarning: invalid value encountered in sqrt
  errs[r] = (z/n[r].astype('float'))*np.sqrt(ns[r].astype('float')*(1.0-ns[r].astype('float')/n[r].astype('float')))
/home/ebert/jupyter-notebooks/FNode/analyses/qdp/QDP/qdp.py:258: RuntimeWarning: divide by zero encountered in divide
  e['iterations'][i]['retention'] = retention/loaded
/home/ebert/jupyter-notebooks/FNode/analyses/qdp/QDP/qdp.py:258: RuntimeWarning: invalid value encountered in divide
  e['iterations'][i]['retention'] = retention/loaded


FigureCanvasNbAgg()